# PostgreSQL Practice

This notebook will contain the work associated with postgresqltutorial.com's python tutorial

Source: http://www.postgresqltutorial.com/postgresql-python/

## Step 1: Create and connect to the 'suppliers' database in terminal

Enter `/Applications/Postgres.app/Contents/Versions/12/bin/psql -p5432 "database_name"` where `database_name` is any existing database or the psql command if you already configured it

In [1]:
# THIS CELL WAS THE ORIGINAL ATTEMPT, IGNORE IT

# Steps adapted from https://stackoverflow.com/a/46877364

# Install psycopg2 if not already on your computer

# Step 1: Attempt to run the following in command line (Python 3):
# sudo apt install libpq-dev python3-dev

# A Java pop-up request may tell you:
# "To use the “apt” command-line tool you need to install a JDK." (Java Developer Kit)
# This is because this package is written in C and adapted for python

# You have a few options for each operating system compressed and uncompressed,
# I selected jdk-13.0.1_osx-x64_bin.dmg
# Run the sudo command after installation 



#!pip install psycopg2

# If you are still having trouble try updating setup tools
# !pip install --upgrade pip setuptools

#sudo -H pip3 install gitsome

In [2]:
# Adapted from https://stackoverflow.com/a/24645416

# You need homebrew for this version

# Step 1: Install Postgresql in terminal:
# brew install postgresql


# Step 2: Install psycopg2
# pip install psycopg2

#Step 3: Install config
# pip install config

In [3]:
import psycopg2
#from config import Config
from config import config

# Adapted from https://stackoverflow.com/a/58136849

In [4]:
# Connect in python to the database by creating a cursor object

# You can specify a "port" parameter, the default port is 5432

# conn = psycopg2.connect("dbname=suppliers user=postgres password=postgres")

# Or:

conn = psycopg2.connect(host="localhost",database="suppliers", user="postgres", password="postgres")

Instead of typing all parameters in the cursor you can use a `.ini` (initialization/configuration) file to store the information. This file will be written in a `postgresql` style format.

A function in the `config.py` file returns the parameters in a psycopg2 friendly format.

In [5]:
params = config()
print(params)

{'host': 'localhost', 'database': 'suppliers', 'user': 'postgres', 'password': 'postgres'}


In [6]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()
 
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
      
        # create a cursor
        cur = conn.cursor()
        
   # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
 
        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
       # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [7]:
# https://stackoverflow.com/questions/419163/what-does-if-name-main-do

# This file is the main file, not the imported config file.

if __name__ == '__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.0 on x86_64-apple-darwin16.7.0, compiled by Apple LLVM version 8.1.0 (clang-802.0.42), 64-bit',)
Database connection closed.


# Step 2: A Function That Executes Commands

In [8]:
# This function will execute commands NOT display results of queries!

def execute_commands(psql_commands):
    
    conn = None
    
    try:
        # read the connection parameters
        # The connection parameters are listed in the files
        # 'config.py' and 'database.ini'
        params = config()
        
        # connect to the PostgreSQL server
        # Info on args vs kwargs: http://book.pythontips.com/en/latest/args_and_kwargs.html
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        
        try:
            # If psql_commands is not a list
            cur.execute(psql_commands)
            
        except:
            # create table one by one
            for command in psql_commands:
                cur.execute(command)

        
        # close communication with the PostgreSQL database server
        cur.close()
        
        # commit the changes
        conn.commit()
        
    # Print error if any portion of the try statement fails
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    # Close the connection 
    finally:
        if conn is not None:
            conn.close()

# Step 3: Create Tables

Source: http://www.postgresqltutorial.com/postgresql-python/create-tables/

In [9]:
# This statement creates four tables within the 'supplies' database

""" create tables in the PostgreSQL database"""
create_table_commands = (
        """
        CREATE TABLE vendors (
            vendor_id SERIAL PRIMARY KEY,
            vendor_name VARCHAR(255) NOT NULL
        )
        """,
        """ CREATE TABLE parts (
                part_id SERIAL PRIMARY KEY,
                part_name VARCHAR(255) NOT NULL
                )
        """,
        """
        CREATE TABLE part_drawings (
                part_id INTEGER PRIMARY KEY,
                file_extension VARCHAR(5) NOT NULL,
                drawing_data BYTEA NOT NULL,
                FOREIGN KEY (part_id)
                REFERENCES parts (part_id)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE vendor_parts (
                vendor_id INTEGER NOT NULL,
                part_id INTEGER NOT NULL,
                PRIMARY KEY (vendor_id , part_id),
                FOREIGN KEY (vendor_id)
                    REFERENCES vendors (vendor_id)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (part_id)
                    REFERENCES parts (part_id)
                    ON UPDATE CASCADE ON DELETE CASCADE
        )
        """)

In [10]:
if __name__ == '__main__':
    execute_commands(create_table_commands)

In [11]:
params = config()
        
# connect to the PostgreSQL server
# Info on args vs kwargs: http://book.pythontips.com/en/latest/args_and_kwargs.html
conn = psycopg2.connect(**params)
cur = conn.cursor()

cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public';""")
print (cur.fetchall())

# for table in cur.fetchall():
#     print(table)
    
cur.close()
conn.commit()


[('parts',), ('part_drawings',), ('vendors',), ('vendor_parts',)]


# Step 4: Insert Data Into A Table


Source: http://www.postgresqltutorial.com/postgresql-python/insert/

The end of the try statement is the only part that is changed within the function in step 2

In [12]:
def insert_vendor(vendor_name):
    """ insert a new vendor into the vendors table """
    sql = """INSERT INTO vendors(vendor_name)
             VALUES(%s) RETURNING vendor_id;"""
    conn = None
    vendor_id = None
    try:
        # read database configuration
        params = config()
        # connect to the PostgreSQL database
        conn = psycopg2.connect(**params)
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        cur.execute(sql, (vendor_name,))
        
        # get the generated id back
        vendor_id = cur.fetchone()[0]
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
 
    return vendor_id

In [13]:
if __name__ == '__main__':
    # insert one vendor
    insert_vendor("3M Co.")

In [14]:
def insert_vendor_list(vendor_list):
    """ insert multiple vendors into the vendors table  """
    sql = "INSERT INTO vendors(vendor_name) VALUES(%s)"
    conn = None
    try:
        # read database configuration
        params = config()
        # connect to the PostgreSQL database
        conn = psycopg2.connect(**params)
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        cur.executemany(sql,vendor_list)
        
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [15]:
if __name__ == '__main__':
    # insert multiple vendors
    insert_vendor_list([
        ('AKM Semiconductor Inc.',),
        ('Asahi Glass Co Ltd.',),
        ('Daikin Industries Ltd.',),
        ('Dynacast International Inc.',),
        ('Foster Electric Co. Ltd.',),
        ('Murata Manufacturing Co. Ltd.',)
    ])

# Step 5: Updating Data

Source: http://www.postgresqltutorial.com/postgresql-python/update/

In [30]:
def query_db(query):
    # read database configuration
    params = config()
    # connect to the PostgreSQL database
    conn = psycopg2.connect(**params)
    # create a new cursor
    cur = conn.cursor()
    # execute the INSERT statement
    cur.execute(query)
    # # get the query results
    query_results = cur.fetchall()
    # commit the changes to the database
    conn.commit()
    # close communication with the database
    cur.close()
    return query_results

In [32]:
def update_vendor(vendor_id, vendor_name):
    """ update vendor name based on the vendor id """
    sql = """ UPDATE vendors
                SET vendor_name = %s
                WHERE vendor_id = %s"""
    conn = None
    updated_rows = 0
    try:
        # read database configuration
        params = config()
        # connect to the PostgreSQL database
        conn = psycopg2.connect(**params)
        # create a new cursor
        cur = conn.cursor()
        # execute the UPDATE  statement
        cur.execute(sql, (vendor_name, vendor_id))
        # get the number of updated rows
        updated_rows = cur.rowcount
        # Commit the changes to the database
        conn.commit()
        # Close communication with the PostgreSQL database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
 
    return updated_rows

In [33]:
query_db("""SELECT * FROM vendors WHERE vendor_id = 1""")

[(1, '3M Co.')]

In [34]:
update_vendor("1", "3M Corp")

1

In [35]:
query_db("""SELECT * FROM vendors WHERE vendor_id = 1""")

[(1, '3M Corp')]

# Step 6: Querying Data


## Part A: with cursor only
Source: http://www.postgresqltutorial.com/postgresql-python/query/

In [58]:
def query_db(query):
    
    query_results = []
    
    try:
        # read database configuration
        params = config()
        # connect to the PostgreSQL database
        conn = psycopg2.connect(**params)
        # create a new cursor
        cur = conn.cursor()

        # execute the INSERT statement
        cur.execute(query)
        print("The number of results: ", cur.rowcount)

        # # get the query results
        query_results = cur.fetchall()

        # close communication with the database
        cur.close()
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    finally:
        if conn is not None:
            conn.close()
    if query_results:
        return query_results


In [61]:
q = query_db("SELECT vendor_id, vendor_name FROM vendors ORDER BY vendor_name")

The number of results:  7


In [62]:
q

[(1, '3M Corp'),
 (2, 'AKM Semiconductor Inc.'),
 (3, 'Asahi Glass Co Ltd.'),
 (4, 'Daikin Industries Ltd.'),
 (5, 'Dynacast International Inc.'),
 (6, 'Foster Electric Co. Ltd.'),
 (7, 'Murata Manufacturing Co. Ltd.')]

## Part B: With Pandas

In [65]:
import pandas as pd

In [63]:
def print_table_contents(table_name, query, database = psycopg2.connect(**params)):
    results = []
    cursor = database.cursor()
    cursor.execute("SELECT {} FROM {};".format(query, table_name))
    results = cursor.fetchall()
    cursor.close()
    database.close()
    return results

In [64]:
print_table_contents("vendors", "*")

[(2, 'AKM Semiconductor Inc.'),
 (3, 'Asahi Glass Co Ltd.'),
 (4, 'Daikin Industries Ltd.'),
 (5, 'Dynacast International Inc.'),
 (6, 'Foster Electric Co. Ltd.'),
 (7, 'Murata Manufacturing Co. Ltd.'),
 (1, '3M Corp')]

In [66]:
def create_pandas_table(sql_query, database = psycopg2.connect(**params)):
    table = pd.read_sql_query(sql_query, database)
    database.close()
    return table

In [67]:
create_pandas_table("SELECT vendor_id, vendor_name FROM vendors ORDER BY vendor_name")

,vendor_id,vendor_name
0,1,3M Corp
1,2,AKM Semiconductor Inc.
2,3,Asahi Glass Co Ltd.
3,4,Daikin Industries Ltd.
4,5,Dynacast International Inc.
5,6,Foster Electric Co. Ltd.
6,7,Murata Manufacturing Co. Ltd.
